<a href="https://colab.research.google.com/github/LuisIrigoyen/trabajo_final_RL/blob/main/proyecto_final_APR_V06_5_en_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False


## 1.3. Montar carpeta de datos local (solo Colab)

In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico  exists.

Colab: Changing directory to  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
Archivos en el directorio: 
['dqn_models', 'checkpoints', 'dqn_05_weights.h5f.data-00000-of-00001', 'dqn_05_weights.h5f.index', 'dqn_06_weights.h5f.index', 'dqn_06_weights.h5f.data-00000-of-00001', 'dqn_07_weights.h5f.index', 'dqn_07_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.index', '.ipynb_checkpoints', 'dqn_09_weights.h5f.index', 'dqn_09_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.index', 'video', 'dqn_06_3_weights.h5f.index', 'dqn_06

## 1.4. Instalar librerías necesarias

In [3]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12  #2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.05
  %pip install Keras==2.2.4
  %pip install tensorflow==2.1.0
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-r1s7xbmy
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-r1s7xbmy
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


## **PARTE 3**. Desarrollo y preguntas
Importar librerías

In [4]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Permute
#from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint


#### Configuración base

In [5]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

#--- nuevo -- ejfm
height, width, channels = env.observation_space.shape
obs = env.reset()
print(height, width, channels)
print("Forma de la observación:", obs.shape)  # Debe ser (height, width, channels)
env.unwrapped.get_action_meanings()

210 160 3
Forma de la observación: (210, 160, 3)


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [13]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

        if batch.ndim == 4:
            batch = np.expand_dims(batch, axis=-1)
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

## 1. EJFM Implementación de la red neuronal

In [14]:
def build_model(height, width, channels, actions):

    model = Sequential()
    #model.add(Permute((2, 3, 1))) # (4, 84, 84, 1) a (84, 84, 4)
    model.add(Conv2D(32, (8, 8), strides=(4, 4),activation='relu', input_shape=(4,84,84,1)))
    model.add(Conv2D(64, (4, 4), strides=(2, 2),activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1),activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

In [15]:
model = build_model(84, 84, 1, nb_actions)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 4, 20, 20, 32)     2080      
                                                                 
 conv2d_10 (Conv2D)          (None, 4, 9, 9, 64)       32832     
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 7, 7, 64)       36928     
                                                                 
 flatten_3 (Flatten)         (None, 12544)             0         
                                                                 
 dense_6 (Dense)             (None, 512)               6423040   
                                                                 
 dense_7 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 6,497,958
Trainable params: 6,497,958
No

## 2. EJFM Implementación de la solución DQN

In [16]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [17]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(),
        attr='eps',
        value_max=1.0,
        value_min=0.1,
        value_test=0.05,
        nb_steps=1000000)

    memory = SequentialMemory(
        limit=200000,
        window_length=4)

    dqn = DQNAgent(model=model,
                   nb_actions=nb_actions,
                   policy=policy,
                   memory=memory,
                   processor=AtariProcessor(),
                   nb_steps_warmup=50000,
                   gamma=0.99,
                   target_model_update=10000,
                   train_interval=4,
                   delta_clip=1.0,
                   enable_double_dqn=True,
                   enable_dueling_network=True,
                   dueling_type='avg',
                   batch_size=32,
                   )
    return dqn

In [18]:
from rl.callbacks import ModelIntervalCheckpoint

# Guardar el modelo cada 10,000 pasos
checkpoint_callback = ModelIntervalCheckpoint(
    filepath='checkpoints/dqn_v06_5_weights_{step}.h5f',
    interval=10000,
    verbose=1
)

In [19]:
import os
os.makedirs("checkpoints", exist_ok=True)

In [20]:
from keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

dqn = build_agent(model, nb_actions)
dqn.compile(RMSprop(learning_rate=0.00025, epsilon=0.01,rho=0.95))


In [ ]:
from datetime import datetime

print(f"⏱ Tiempo de inicio: {datetime.now().strftime('%H:%M:%S')}")

dqn.fit(env, nb_steps=500000, visualize=False, verbose=2,callbacks=[checkpoint_callback])

print(f"⏱ Tiempo de fin: {datetime.now().strftime('%H:%M:%S')}")

# Guardando modelo
dqn.save_weights('dqn_v06_5_weights.h5f')


⏱ Tiempo de inicio: 23:26:13
Training for 300000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    883/300000: episode: 1, duration: 7.183s, episode steps: 883, steps per second: 123, episode reward: 14.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.571 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   1519/300000: episode: 2, duration: 4.287s, episode steps: 636, steps per second: 148, episode reward:  7.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.495 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   2239/300000: episode: 3, duration: 5.888s, episode steps: 720, steps per second: 122, episode reward:  9.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.439 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   2753/300000: episode: 4, duration: 3.342s, episode steps: 514, steps per second: 154, episode reward:  6.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.558 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   3399/300000: episode: 5, duration: 4.137s, episode steps: 646, steps per second: 156, episode reward:  7.000,

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  10487/300000: episode: 15, duration: 46.674s, episode steps: 590, steps per second:  13, episode reward:  7.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.383 [0.000, 5.000],  loss: 0.006486, mean_q: 0.053068, mean_eps: 0.953902
  11578/300000: episode: 16, duration: 99.048s, episode steps: 1091, steps per second:  11, episode reward: 18.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.440 [0.000, 5.000],  loss: 0.006776, mean_q: 0.039241, mean_eps: 0.950356
  11970/300000: episode: 17, duration: 36.252s, episode steps: 392, steps per second:  11, episode reward:  7.000, mean reward:  0.018 [ 0.000,  1.000], mean action: 2.658 [0.000, 5.000],  loss: 0.009222, mean_q: 0.044636, mean_eps: 0.947017
  13087/300000: episode: 18, duration: 101.574s, episode steps: 1117, steps per second:  11, episode reward: 12.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.592 [0.000, 5.000],  loss: 0.007270, mean_q: 0.043998, mean_eps: 0.943624
  14465/300000: episode: 19, 

In [ ]:
# Testing part to calculate the mean reward
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
weights_filename = 'dqn_v06_5_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 22.000, steps: 916
Episode 2: reward: 7.000, steps: 402
Episode 3: reward: 6.000, steps: 379
Episode 4: reward: 8.000, steps: 510
Episode 5: reward: 9.000, steps: 617
Episode 6: reward: 10.000, steps: 646
Episode 7: reward: 8.000, steps: 440
Episode 8: reward: 9.000, steps: 696
Episode 9: reward: 6.000, steps: 385
Episode 10: reward: 3.000, steps: 364


In [ ]:
from google.colab import drive
import shutil
import glob  # <-- ¡Falta este import!
import os

# 1. Montar Google Drive
#drive.mount('/content/drive')

# 2. Crear directorio en Drive si no existe
os.makedirs('/content/gdrive/MyDrive/Colab_Outputs', exist_ok=True)

# 3. Buscar y copiar todos los archivos coincidentes
archivos = glob.glob('dqn_v06_5_weights*')  # Encuentra todos los archivos con ese patrón

for archivo in archivos:
    shutil.copy(
        archivo,
        f'/content/gdrive/MyDrive/Colab_Outputs/{os.path.basename(archivo)}'
    )
    print(f"¡Archivo {archivo} copiado a Drive!")

# 4. Buscar y copiar todos los archivos coincidentes
archivos = glob.glob('checkpoints/dqn_v06_5_weights*')  # Encuentra todos los archivos con ese patrón

for archivo in archivos:
    shutil.copy(
        archivo,
        f'/content/gdrive/MyDrive/Colab_Outputs/{os.path.basename(archivo)}'
    )

print("✅ ¡Todos los archivos copiados!")

¡Archivo dqn_v06_4_weights.h5f.data-00000-of-00001 copiado a Drive!
¡Archivo dqn_v06_4_weights.h5f.index copiado a Drive!


OSError: [Errno 107] Transport endpoint is not connected